# Image segmentaion with masked facies

Squish rectangular images to square

Using mask images processed in 01_02_mask_processing

1100 x 1100 size

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
from numbers import Integral
from random import uniform
from PIL import Image as pil_image
import fastai
from fastai.vision import *
from fastai.vision import Image
from fastai.vision.transform import _minus_epsilon
from fastai.vision.data import SegmentationProcessor
from fastai.vision.interpret import SegmentationInterpretation
from fastai.callbacks.tracker import SaveModelCallback
from mask_functions import *
from collections import defaultdict
import cv2
from IPython.display import display 
import datetime
import uuid
from scipy.stats import mode

In [3]:
fastai.__version__

'1.0.60.dev0'

In [4]:
#torch.cuda.set_device(2)

In [5]:
DATE = datetime.datetime.today().strftime('%Y%m%d')
UID=str(uuid.uuid4())[:8]
print(f'UID: {UID}, DATE: {DATE}')    

UID: d287a05e, DATE: 20191217


In [6]:
NB='02_00'

In [7]:
SUBSET_DATA=False
SUBSET_LEN=171

In [8]:
tgt_height = 550
data_dir = Path('data')
train_images = data_dir/'train_images'
test_img = train_images/'mask_fill/test'

train_path = train_images/'mask_fill/train'
train_mask = train_path/'masks'
train_img = train_path/'images'


#subset
train_sub_path = train_images/'mask_fill/subset/train'
train_sub_mask = train_sub_path/'masks'
train_sub_img = train_sub_path/'images'

#train_path = train_sub_path
#train_mask = train_sub_mask
#train_img = train_sub_img

train_images = data_dir/'train_images'
test_img = train_images/'mask_fill/test'

cropped_base=train_images/'cropped'
cropped_base = cropped_base/'mask_fill'
cropped_train = cropped_base/'train'
cropped_train_mask = cropped_train/'masks'
cropped_train_img = cropped_train/'images'

cropped_test = cropped_base/'test'
cropped_test_img=cropped_test/'cropped'


In [9]:
filename = 'data/CAX_LogFacies_Train_File.csv'

In [10]:
file_test = 'data/CAX_LogFacies_Test_File.csv'

#### all data

In [11]:
training_data = pd.read_csv(filename)
training_data.head()
training_data['well_file']='well_'+training_data['well_id'].astype(str)+'.png'
wells=training_data['well_file'].unique()
all_wells_df=pd.DataFrame(wells)
all_wells_df.head()
    
df_val = all_wells_df.sample(frac=0.2)
idx=df_val.index
df_trn=all_wells_df[~all_wells_df.index.isin(idx)]
assert len(df_val)+len(df_trn)==len(all_wells_df)
#df_val.to_csv(train_path/'val_20pct.csv', index=False, header=False)

In [12]:
def gen_splits():
    for i in range(6):
        df_val = all_wells_df.sample(frac=0.2)
        idx=df_val.index
        df_trn=all_wells_df[~all_wells_df.index.isin(idx)]
        assert len(df_val)+len(df_trn)==len(all_wells_df)
        df_val.to_csv(train_path/f'val_20pct_{i}.csv', index=False, header=False)

In [13]:
#gen_splits()


In [29]:
def get_well_names_from_images(images):
    names=[]
    train_img_names = get_image_files(images)
    for n in train_img_names:
        p = n.name.split('.png')[0]
        names.append(p)
    return names

In [32]:
def gen_crop_splits(img_list):
    random.shuffle(img_list)
    splits=np.array_split(img_list, 5)
    for i, s in enumerate(splits):
        df = pd.DataFrame(s,columns=['well_id'])
        df.well_id = df.well_id + '.png'
        df.to_csv(cropped_train/f'val_20pct_{i}.csv', index=False, header=False)

In [33]:
well_train_images=get_well_names_from_images(cropped_train_img)
gen_crop_splits(well_train_images)
